In [1]:
import json
import tensorflow as tf
import numpy as np
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense

batch_size = 64  
epochs = 100  
latent_dim = 256  
num_samples = 5000
num_test = 100

data_path = 'translation2019zh_train.json'

train_texts = []
test_data = []
input_texts = []
target_texts = []

input_file = open(data_path, encoding = 'utf8')
for i,line in enumerate(input_file.readlines(),1):
    if i > num_samples + num_samples:
        break
    elif i > num_samples:
        test_data.append(json.loads(line))
    else:
        train_texts.append(json.loads(line))
input_file.close()

input_characters = set()
target_characters = set()

for train_text in train_texts:
    input_text = train_text['english']
    target_text = '\t' + train_text['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save('s2s.h5')

Number of samples: 5000
Number of unique input tokens: 319
Number of unique output tokens: 3528
Max sequence length for inputs: 282
Max sequence length for outputs: 150
Epoch 1/100
63/63 [==============================] - 274s 4s/step - loss: 1.6824 - val_loss: 1.6831
Epoch 2/100
63/63 [==============================] - 221s 4s/step - loss: 1.6460 - val_loss: 1.6769
Epoch 3/100
63/63 [==============================] - 215s 3s/step - loss: 1.6414 - val_loss: 1.6756
Epoch 4/100
63/63 [==============================] - 222s 4s/step - loss: 1.6376 - val_loss: 1.6739
Epoch 5/100
63/63 [==============================] - 220s 4s/step - loss: 1.6363 - val_loss: 1.6719
Epoch 6/100
63/63 [==============================] - 225s 4s/step - loss: 1.6353 - val_loss: 1.6709
Epoch 7/100
63/63 [==============================] - 219s 3s/step - loss: 1.6347 - val_loss: 1.6692
Epoch 8/100
63/63 [==============================] - 226s 4s/step - loss: 1.6342 - val_loss: 1.6718
Epoch 9/100
63/63 [============

63/63 [==============================] - 226s 4s/step - loss: 1.6167 - val_loss: 1.6576
Epoch 81/100
63/63 [==============================] - 232s 4s/step - loss: 1.6169 - val_loss: 1.6581
Epoch 82/100
63/63 [==============================] - 281s 4s/step - loss: 1.6160 - val_loss: 1.6596
Epoch 83/100
63/63 [==============================] - 225s 4s/step - loss: 1.6153 - val_loss: 1.6669
Epoch 84/100
63/63 [==============================] - 226s 4s/step - loss: 1.6152 - val_loss: 1.6642
Epoch 85/100
63/63 [==============================] - 230s 4s/step - loss: 1.6139 - val_loss: 1.6584
Epoch 86/100
63/63 [==============================] - 231s 4s/step - loss: 1.6138 - val_loss: 1.6604
Epoch 87/100
63/63 [==============================] - 236s 4s/step - loss: 1.6129 - val_loss: 1.6572
Epoch 88/100
63/63 [==============================] - 232s 4s/step - loss: 1.6134 - val_loss: 1.6594
Epoch 89/100
63/63 [==============================] - 225s 4s/step - loss: 1.6122 - val_loss: 1.6581
Epo

In [15]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

test_texts = []
test_characters = set()
for text in test_data:
    test_texts.append(text['english'])
    for char in text['english']:
        if char not in test_characters:
            test_characters.add(char)
            
test_characters = sorted(list(test_characters))
num_test = len(test_characters)
test_index = dict([(char, i) for i, char in enumerate(test_characters)])
test_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

In [ ]:
for seq_index in range(len(test_data)):
    input_seq = test_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(seq_index + 1, '.')
    print('English:', test_data[seq_index])
    try:
        print('Chinese:', decoded_sentence)
    except:
        print('Chinese:', decoded_sentence.encode('ascii', 'replace'))

print(test_data)

1 .
English: {'english': 'But in before this long reaches for several months to rescue in the city or not argument, the supervising and managing level has not said a word.', 'chinese': '而在此前长达数月的救市与否的争论中，监管层未发一言。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
2 .
English: {'english': 'With their help, all the apples were picked and put back on the cart.', 'chinese': '侥幸的是，苏和汽车司机都没受伤。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的
3 .
English: {'english': "Crohn's disease (CD) is a chronic transmural inflammatory disease that may involve any part of the alimentary tract from mouth to anus.", 'chinese': '克隆氏病症是一种可侵犯消化道任何一部份的慢性全肠壁层侵犯的发炎疾病。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，

24 .
English: {'english': 'On January 23, thousands of Bangladeshi Muslims boarded overcrowded trains as they tried to return home after attending the three-day World Islamic Congregation in Dhaka, Bangladesh.', 'chinese': '1月23日，在孟加拉国首都达卡，数千名穆斯林教徒参加完为期三天的世界穆斯林代表大会后，挤火车回家。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
25 .
English: {'english': 'Handlers that do not process, repack or re-label organic products do not need to be certified.', 'chinese': '不对有机产品进行加工、不换包装或不换标签的贸易商不必认证。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
26 .
English: {'english': 'Several studies were commissioned, and a variety of panels heard expert witnesses and took testimony about the need for reform.', 'chinese': '有关方面围绕改革的需要展开了数项研究，举行了多种专题会议，听取专家意见和举行听证。'}
Chinese: 我们们，，，，，，，，，

47 .
English: {'english': 'The results indicated that the ICA in three algorithms JADE, MSNR, GOSA can be extracted to structure mode, which had a good consistency with the modal parameters from the OMA.', 'chinese': '研究结果表明ICA中的JADE、MSNR、GOSA三种算法均可提取试验模型的结构模态参数，它与OMA提取的模态参数具有良好的一致性。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
48 .
English: {'english': 'Would you like to go bicycling with me in the mountains?', 'chinese': '要不要和我—起去山里骑自行车？'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
49 .
English: {'english': "Men's ability to turn a sniffle into flu and a headache into a migraine has long been a source of irritation to wives and girlfriends.", 'chinese': '男性将流鼻涕转变成流感，将轻微头痛转变成偏头痛的“能力”，长期以来一直是激怒他们妻子和女朋友的一个根源。 （因为普遍认为女性在流感面前更脆弱。）'}
Chinese: 我们们，，，，，，，，，，，，

70 .
English: {'english': 'The stabilizing effect of hot-electron and line-tying on the low-frequency plasma interchange mode is analysed.', 'chinese': '分析了热电子与线捆对低频等离子体交换模的稳定作用。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
71 .
English: {'english': 'So I think there are very many similarities between what’s happening in Europe and what’s happening in the Arab world.', 'chinese': '因此，我认为，欧洲所发生的事情和阿拉伯世界所发生的动荡之间有很多相似之处。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
72 .
English: {'english': 'When he fell from the cliff, he beat his brains out.', 'chinese': '他从悬崖上跌了下去， 头破血流而死。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
73 .
English: {'engli

92 .
English: {'english': 'Black hole of algorithms, and the dichotomy, I wrote some satay to know that they feel can also.', 'chinese': '黑洞的算法，和二分法，我写的，嗲给知道一些，自己感觉还可以。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
93 .
English: {'english': 'if the time period of paying premiums is not less than ten years, the maximum period of receiving unemployment insurance compensation is 24 months;', 'chinese': '累计缴费时间10年以上的，领取失业保险金的期限最长为24个月。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
94 .
English: {'english': 'Inverse triangle: a similar heart-shaped, the amount of large, small jaw, are one of the ideal type face, all the collars are suitable.', 'chinese': '逆三角形：类似心形，上额宽大，下颚狭小，是属于理想型脸型之一，所有的领子都适合。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，

115 .
English: {'english': 'Where is my bus?', 'chinese': '我的巴士在哪里？'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
116 .
English: {'english': 'Increased purchasing power. The FBMP increases the purchasing power of First Book, enabling it over time to leverage additional discounts for all books it purchases.', 'chinese': '提高了购买力。第一书市提高了第一本书的购买力，从而平衡了其购买的所有书籍的额外折扣。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
117 .
English: {'english': 'The scholarship and research grants distribution is as appendix table 1.', 'chinese': '奖学金和奖教金具体分布详请见附件1。'}
Chinese: 我们们，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的，，的，，的，，的，，的，的，的的的的的的，的的，的，的的，的的，的的，的，的的
118 .
English: {'english': 'I met a lot of friends, a lot 